In [ ]:
import (
	"database/sql"
	"fmt"
	"io/fs"
    "io/ioutil"
	"net/url"
    "os"
	"path/filepath"
	"regexp"
    "strconv"
    "strings"

	_ "github.com/lib/pq"
)

type Config struct {
	Database string
	Host     string
	Port     int
	SslMode  string
}

type Client struct {
	Connection *sql.DB
}

func (c *Config) NewClient(password, username *string) (*Client, error) {
	var client Client
	var err error

	if (password != nil) && (username != nil) {
		denodoConnUrl := fmt.Sprintf(
			"postgres://%s:%s@%s:%d/%s?sslmode=%s",
			*username,
			url.QueryEscape(*password),
			c.Host,
			c.Port,
			c.Database,
			c.SslMode,
		)

		client.Connection, err = sql.Open("postgres", denodoConnUrl)
		if err != nil {
			return nil, err
		}
	}

	return &client, nil
}
func (c *Client) ResultSet(sqlStmt *string) ([][]string, error) {
	var err error
	var results *sql.Rows

	results, err = c.Connection.Query(*sqlStmt)
	if err != nil {
		return nil, err
	}

	columns, err := results.Columns()
	if err != nil {
		return nil, err
	}

	count := len(columns)
	tableData := [][]string{}
	values := make([]interface{}, count)
	valuePtrs := make([]interface{}, count)
    tableData = append(tableData, columns)
	for results.Next() {
		for i := 0; i < count; i++ {
			valuePtrs[i] = &values[i]
		}
		results.Scan(valuePtrs...)
		record := make([]string, count)
		for i := range columns {
			switch values[i].(type) {
			case nil:
				var s string
				record[i] = s
			case []byte:
				record[i] = string(values[i].([]byte))
			default:
				record[i] = fmt.Sprintf("%s", values[i])
			}
		}
		tableData = append(tableData, record)
	}

	return tableData, nil
}

func (c *Client) ExecuteSQL(sqlStmt *string) error {
	var err error

	_, err = c.Connection.Exec(*sqlStmt)
	if err != nil {
		return err
	}

	return err
}


In [ ]:
var config *Config
var client *Client
var err error
var password string
var sqlStmt string
var username string

password = os.Getenv("TF_VAR_denodo_password")
sqlStmt = "CALL GET_ELEMENTS('l_kpmc_mars',NULL,'Views');"
username = os.Getenv("TF_VAR_denodo_username")

config = new(Config)
config.Database = "admin"
config.Host = os.Getenv("TF_VAR_denodo_host")
config.Port, err = strconv.Atoi(os.Getenv("TF_VAR_denodo_port"))
config.SslMode = "require"

client, err = config.NewClient(&password, &username)
if err != nil {
    fmt.Println(err)
}
data, err := client.ResultSet(&sqlStmt)
if err != nil {
    fmt.Println(err)
}

In [ ]:
data[0]

In [ ]:
fmt.Println(data[0][0])
fmt.Println(data[0][1])
fmt.Println(data[0][2])
fmt.Println(data[0][3])
fmt.Println(data[0][4])
fmt.Println(data[0][5])
fmt.Println(data[0][6])
fmt.Println(data[0][7])
fmt.Println(data[0][8])
fmt.Println(data[0][9])
fmt.Println(data[0][10])
fmt.Println(data[0][11])

In [ ]:
contents, _ := ioutil.ReadFile("create_view.sql")
string(contents)

In [ ]:
check := regexp.MustCompile(`(?i)CREATE( OR REPLACE)? VIEW\s+\w{1,}\s+AS`)

In [ ]:
check.FindAllStringSubmatch(string(contents),-1)

In [ ]:
re := regexp.MustCompile(`(?i)(CREATE|OR|RELACE|VIEW|AS|\s)`)
re.ReplaceAllString(output[0],"")

In [ ]:
files, err := ioutil.ReadDir("dervived_view")
if err != nil {
    fmt.Println(err)
}

for _, file := range files {
    fileExtension := filepath.Ext(file)
    fmt.Println(file.Name())
}


In [ ]:
err = filepath.Walk(".", func(path string, info os.FileInfo, err error) error {
    if err != nil {
        return err
    }
    if ! info.IsDir() {
        if filepath.Ext(path) == ".sql" {
            contents, err := ioutil.ReadFile(path)
            if err != nil {
                return err
            }
            check := regexp.MustCompile(`(?i)CREATE( OR REPLACE)? VIEW\s+\w{1,}`)
            getName := regexp.MustCompile(`(?i)(CREATE|OR|REPLACE|VIEW|\s)`)
            
            commands := strings.Split(string(contents), ";")
            fmt.Println(commands)
            for _, command := range commands {
                if check.MatchString(command){
                    fmt.Println(path)
                    fmt.Println(command)
                } 
            } 
        }
    }
    return err
})

In [ ]:
var fileData []byte
var contents string
var name string
var database string
err = filepath.Walk(".", func(path string, info os.FileInfo, err error) error {
    database = "test_database"
    if !info.IsDir() {
        if filepath.Ext(path) == ".sql" {
            fileData, err = ioutil.ReadFile(path)
            if err != nil {
                return err
            }
            contents = string(fileData)
            for _, command := range strings.Split(contents, ";") {
                checkDerivedView := regexp.MustCompile(`(?i)CREATE( OR REPLACE)? VIEW\s+\w{1,}`)
                if checkDerivedView.MatchString(command) {
                    getName := regexp.MustCompile(`(?i)(CREATE|OR|REPLACE|VIEW|AS|\s)`)
                    subString := checkDerivedView.FindString(command)

                    name = getName.ReplaceAllString(subString, "")
                    sqlStmt = fmt.Sprintf(
                        `CONNECT DATABASE %s;
                        DROP VIEW IF EXISTS %s CASCADE;`,
                        database,
                        name,
                    )
                    fmt.Println(sqlStmt)
                }
            }
        }
    }
    return err
})